# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [1]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023
import sys
# colab seems to require an explicit path for this to work from mounted drive
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')

Mounted at /content/gdrive
/content/gdrive/MyDrive/CITS4012_2023


In [ ]:
# when not using colab
import sys
sys.path.append('./src/QA')
import reload_recursive
%reload data_prep

In [ ]:
from data_prep import DataPrep

In [ ]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_doc_raw_train = DataPrep.convert_pd_to_json(df)

In [ ]:
question_doc_raw_train["Q1"]

{'document': [('A partly submerged glacier cave on Perito Moreno Glacier .',
   0),
  ('The ice facade is approximately 60 m high', 0),
  ('Ice formations in the Titlis glacier cave', 0),
  ('A glacier cave is a cave formed within the ice of a glacier .', 1),
  ('Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice.',
   0)],
 'question': 'how are glacier caves formed?'}

NER tagging, some initial testing of the function

Questions that NER tagging works in:
- how many series in hockey before the stanley cup
- how much are the harry potter movies worth

Questions it doesn't work in:
- how did anne frank die
- what did benedict arnold die
- how old is kurt cobain's daughter


In [ ]:
%reload word_embed
from word_embed import WordEmbed

In [ ]:
%reload feat_extract
from feat_extract import FeatExt

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
%reload embed_doc
from embed_doc import EmbedAndConcat

In [ ]:
# returns a tuple with:
# 0 - question tokenized
# 1 - list of sentences tokenized
# 2 - list of corresponding sentence in-or-out tokens
q1_toks = DataPrep.tokenize_question_and_doc(question_doc_raw_train["Q1"])

In [ ]:
# returns a list of embedded sentences, with each sentence having one array for each word
toks_embedded = EmbedAndConcat.doc_concat(q1_toks[1])

In [ ]:
# same as above, but includes named entity recognition tags in the arrays
doc_ner_tags = FeatExt.ner_tag_doc(question_doc_raw_train["Q1"]["document"])
toks_embedded_with_ner = EmbedAndConcat.doc_concat(q1_toks[1], with_ner=True, doc_ner_tags=doc_ner_tags)

In [ ]:
# these functions do the same, but for questions
q_embedded = EmbedAndConcat.q_concat(q1_toks[0])

In [ ]:
q_ner_tags = FeatExt.ner_tag(question_doc_raw_train["Q1"]["question"])
q_embedded_with_ner = EmbedAndConcat.q_concat(q1_toks[0], with_ner=True, question_ner_tags=q_ner_tags)

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [ ]:
from word_embed import WordEmbed
from feat_extract import FeatExt
from embed_doc import EmbedAndConcat

[=================================================-] 99.4% 127.4/128.1MB downloaded

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
full_embedded_input = {}
# "question" = list of np.arrays for each word
# "document" = a list of embedded sentences, with each sentence having one array for each word
# "answer" = list of sentences, each sentence a list of answer labels
for q_id in question_id:
  try:
    tokens = DataPrep.tokenize_question_and_doc(question_id[q_id])
    answer = tokens[2]
    q_ner_tags = FeatExt.ner_tag(question_id[q_id]["question"])
    q_embedded = EmbedAndConcat.q_concat(tokens[0], with_ner=True, question_ner_tags=q_ner_tags)
    doc_ner_tags = FeatExt.ner_tag_doc(question_id[q_id]["document"])
    doc_embedded = EmbedAndConcat.doc_concat(tokens[1], with_ner=True, doc_ner_tags=doc_ner_tags)
    full_embedded_input[q_id] = {"question": q_embedded, "document": doc_embedded, "answer": answer}
  except:
    print(q_id)
    break

In [ ]:
no_ner_embedded_input = {}
for q_id in question_id:
    tokens = DataPrep.tokenize_question_and_doc(question_id[q_id])
    answer = tokens[2]
    q_embedded = EmbedAndConcat.q_concat(tokens[0])
    doc_embedded = EmbedAndConcat.doc_concat(tokens[1])
    full_embedded_input[q_id] = {"question": q_embedded, "document": doc_embedded, "answer": answer}

In [ ]:
# INSERT CODE FOR BATCH TRAINING

In [2]:
# looking at results file for initial exploration
%cd /content/gdrive/MyDrive/CITS4012_2023/models
import pickle
import pandas as pd
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)
resultkeys = results.keys()
ResultsDict = {}
mod_num = 0
for i in resultkeys:
    mod_num += 1
    mod_dict = {}
    for j in i:
        mod_dict[j[0]] = j[1]
    for k in results[i]:
        for l in ['precision', 'recall', 'f1-score', 'support']:
            mod_dict[f'{k} {l}'] = results[i][k]['test_report']['1'][l]
            mod_dict[f'{k} training {l}'] = results[i][k]['train_report']['1'][l]
    ResultsDict[f'Model {mod_num}'] = mod_dict
resDFBA = pd.DataFrame(ResultsDict) # convert into pandas dataframe
resDFT = resDFBA.transpose()

/content/gdrive/MyDrive/CITS4012_2023/models


In [15]:
import math
print(f'Max F1 for 20 iterations {max(resDFT["20 f1-score"])}')
print(f'Max F1 for 10 iterations {max(resDFT["10 f1-score"])}')
print(f'Max F1 for 5 iterations {max(resDFT["5 f1-score"])}')
print(f'Max F1 for 1 iterations {max(resDFT["1 f1-score"])}')

Max F1 for 20 iterations 0.18330762913854226
Max F1 for 10 iterations 0.17609118935583318
Max F1 for 5 iterations 0.16743569480204962
Max F1 for 1 iterations 0.14067188757959453


In [7]:
margin = 0.005
# find all models that had test F1 scores within margin of the maximum
best_mods_20iter = resDFT[max(resDFT['20 f1-score'])-margin <= resDFT['20 f1-score']]
best_mods_20iter = best_mods_20iter.sort_values(by='20 f1-score', ascending = False)

In [8]:
# view parameters of these models
best_mods_20iter[['doc_with_wm', 'attention_type', 'q_with_ner', 'doc_with_ner', 'hidden_size', 'hidden_type', 'doc_hidden_layers', 'doc_with_pos', 'learning_rate', 'q_with_pos', 'befaft', 'q_cut_size', 'doc_with_tfidf', 'doc_cut_size', 'bidirectional']]

,doc_with_wm,attention_type,q_with_ner,doc_with_ner,hidden_size,hidden_type,doc_hidden_layers,doc_with_pos,learning_rate,q_with_pos,befaft,q_cut_size,doc_with_tfidf,doc_cut_size,bidirectional
Model 390,False,Dot Product,True,True,100,RNN,1,False,0.1,False,False,Max,True,256,False
Model 1337,True,Scaled Dot Product,True,True,100,RNN,1,False,0.1,False,False,Max,True,256,True
Model 1578,False,Cosine,True,False,100,RNN,1,False,0.1,False,False,Max,True,256,True
Model 1840,True,Cosine,False,False,100,RNN,1,False,0.1,False,False,Max,False,256,False
Model 1678,True,Cosine,False,True,100,RNN,1,False,0.1,False,False,Max,False,256,True
Model 559,False,Dot Product,False,True,100,RNN,1,False,0.1,False,False,Max,False,256,True
Model 520,True,Dot Product,False,True,100,RNN,1,False,0.1,False,False,Max,False,256,False
Model 1677,True,Scaled Dot Product,False,True,100,RNN,1,False,0.1,False,False,Max,False,256,True


In [9]:
# get performance metrics for those
best_mods_20iter[['20 precision','20 recall','20 f1-score','20 support']]

,20 precision,20 recall,20 f1-score,20 support
Model 390,0.545346,0.110169,0.183308,41536
Model 1337,0.494101,0.112178,0.182845,36959
Model 1578,0.623764,0.106255,0.181578,49259
Model 1840,0.695626,0.103253,0.179816,56531
Model 1678,0.679418,0.103579,0.179754,55040
Model 559,0.582291,0.106146,0.17956,46031
Model 520,0.447027,0.111527,0.178517,33633
Model 1677,0.602074,0.104772,0.178484,48219


In [10]:
# get f1 scores for the different permutations of NER and POS with all other variables kept the same as best RNN IO model
dpY_qpY_dnY_qnY = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpY_qpY_dnN_qnN = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpN_dnY_qnY = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpN_dnN_qnN = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]

dpY_qpN_dnY_qnY = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpY_qpN_dnN_qnN = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpY_dnY_qnY = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpY_dnN_qnN = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]

dpY_qpY_dnY_qnN = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpY_qpY_dnN_qnY = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpN_dnY_qnN = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpN_dnN_qnY = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]

dpY_qpN_dnY_qnN = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpY_qpN_dnN_qnY = resDFT[(resDFT['doc_with_pos']==True) & (resDFT['q_with_pos']==False) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpY_dnY_qnN = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==False) & (resDFT['doc_with_ner']==True) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]
dpN_qpY_dnN_qnY = resDFT[(resDFT['doc_with_pos']==False) & (resDFT['q_with_pos']==True) & (resDFT['q_with_ner']==True) & (resDFT['doc_with_ner']==False) & (resDFT['doc_with_wm']==False) & (resDFT['attention_type']=='Dot Product') & (resDFT['doc_with_tfidf']==True) & (resDFT['learning_rate']==0.1)& (resDFT['bidirectional']==False)][['20 f1-score']]['20 f1-score'][0]


pos_ner_table = pd.DataFrame([[dpN_qpN_dnN_qnN, dpY_qpN_dnN_qnN, dpN_qpY_dnN_qnN, dpY_qpY_dnN_qnN],[dpN_qpN_dnY_qnN, dpY_qpN_dnY_qnN, dpN_qpY_dnY_qnN, dpY_qpY_dnY_qnN],[dpN_qpN_dnN_qnY, dpY_qpN_dnN_qnY, dpN_qpY_dnN_qnY, dpY_qpY_dnN_qnY],[dpN_qpN_dnY_qnY, dpY_qpN_dnY_qnY, dpN_qpY_dnY_qnY, dpY_qpY_dnY_qnY]], columns=['No POS', 'Doc POS', 'Q POS', 'Doc and Q POS'], index=['No NER', 'Doc NER', 'Q NER','Doc and Q NER'])
pos_ner_table

,No POS,Doc POS,Q POS,Doc and Q POS
No NER,0.135852,0.136478,0.136189,0.135272
Doc NER,0.152413,0.162322,0.157546,0.148523
Q NER,0.156667,0.168905,0.134523,0.145634
Doc and Q NER,0.183308,0.162952,0.154259,0.139319


Before-Answer-After classification models

In [16]:
# Do similar exploration of before-answer-after models
%cd /content/gdrive/MyDrive/CITS4012_2023/models
import pickle
import pandas as pd
with open('RNNBAresults.pkl', 'rb') as f:
    BAresults = pickle.load(f)
resultkeys = BAresults.keys()
ResultsDict = {}
mod_num = 0
for i in resultkeys:
    mod_num += 1
    mod_dict = {}
    for j in i:
        mod_dict[j[0]] = j[1]
    for k in BAresults[i]:
        for l in ['precision', 'recall', 'f1-score', 'support']:
            mod_dict[f'{k} {l}'] = BAresults[i][k]['test_report']['1'][l]
            mod_dict[f'{k} training {l}'] = BAresults[i][k]['train_report']['1'][l]
    ResultsDict[f'Model {mod_num}'] = mod_dict
BAresDFBA = pd.DataFrame(ResultsDict) # convert into pandas dataframe
BAresDFT = BAresDFBA.transpose()

/content/gdrive/MyDrive/CITS4012_2023/models


In [18]:
import math
print(f'Max F1 for 20 iterations {max(BAresDFT["20 f1-score"])}')
print(f'Max F1 for 10 iterations {max(BAresDFT["10 f1-score"])}')
print(f'Max F1 for 5 iterations {max(BAresDFT["5 f1-score"])}')
print(f'Max F1 for 1 iterations {max(BAresDFT["1 f1-score"])}')

Max F1 for 20 iterations 0.19500953985677827
Max F1 for 10 iterations 0.18225309820088048
Max F1 for 5 iterations 0.17684063373718545
Max F1 for 1 iterations 0.1374613229828448


In [20]:
margin = 0.01
BAbest20iter = BAresDFT[max(BAresDFT['20 f1-score'])-margin <= BAresDFT['20 f1-score']]
BAbest20iter = BAbest20iter.sort_values(by='20 f1-score', ascending = False)

In [21]:
BAbest20iter[['doc_with_wm', 'attention_type', 'q_with_ner', 'doc_with_ner', 'hidden_size', 'hidden_type', 'doc_hidden_layers', 'doc_with_pos', 'learning_rate', 'q_with_pos', 'befaft', 'q_cut_size', 'doc_with_tfidf', 'doc_cut_size', 'bidirectional']]

,doc_with_wm,attention_type,q_with_ner,doc_with_ner,hidden_size,hidden_type,doc_hidden_layers,doc_with_pos,learning_rate,q_with_pos,befaft,q_cut_size,doc_with_tfidf,doc_cut_size,bidirectional
Model 591,True,Cosine,True,False,100,RNN,1,False,0.1,True,True,Max,False,256,True
Model 475,False,Dot Product,False,True,100,RNN,1,False,0.1,False,True,Max,True,256,True
Model 510,True,Cosine,True,False,100,RNN,1,False,0.1,False,True,Max,True,256,False
Model 430,True,Dot Product,False,True,100,RNN,1,False,0.1,False,True,Max,True,256,False


In [22]:
BAbest20iter[['20 precision','20 recall','20 f1-score','20 support']]

,20 precision,20 recall,20 f1-score,20 support
Model 591,0.468955,0.1231,0.19501,31966
Model 475,0.449529,0.122758,0.192852,30727
Model 510,0.594089,0.1131,0.190024,44076
Model 430,0.551305,0.113185,0.187812,40871


# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 